<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Computaci%C3%B3n%20cu%C3%A1ntica%20adiab%C3%A1tica%20y%20Quantum%20Annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-optimization qiskit_machine_learning qiskit-ibm-runtime pylatexenc

In [ ]:
import numpy as np
from math import sqrt

# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator, StatevectorSimulator

# import basic plot tools
from qiskit.visualization import plot_histogram

# Funciones auxiliares

# Función para obtener y mostrar el vector de estado
def obten_estado(qcirc, etiqueta="|\psi\!\!> = ", bloch=False):
  estado = Statevector.from_instruction(qcirc)
  display(estado.draw('latex', prefix=etiqueta, max_size=2**qc.num_qubits))
  if bloch:
    display(estado.draw('bloch'))

# **Computación cuántica adiabática y Quantum Annealing**

### Contenidos

1. [Computación cuántica adiabática](#adiabatica)
1. [Quantum Annealing](#annealing)
1. [Discretización de la evolución adiabática (*Trotterization*)](#trotter)
1. [Quantum Approximate Optimization Algorithm (QAOA)](#qaoa)
1. [Variational Quantum Eigensolvers (VQE)](#vqe)


<a name="adiabatica"></a>

## **Computación cuántica adiabática**

Forma alternativa de computación cuántica basada en la evolución temporal de un Hamiltoniano y en el [teorema adiabático](https://en.wikipedia.org/wiki/Adiabatic_theorem)

#### Ecuación de Schrödinger

La evolución temporal de un sistema cuántico viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

$H(t)$ es el Hamiltoniano del sistema en el instante $t$ y define la energía del mismo.

#### Teorema adiabático
Si un sistema cuántico está en un estado base (estado de menor energía o _ground state_) de un Hamiltoniano inicial y se le hace evolucionar muy lentamente hacia un Hamiltoniano final, este estado inicial evolucionará al estado base del Hamiltoniano final, tras un tiempo suficientemente largo.

Supongamos que tenemos un Hamiltoniano $H_c$ (_Hamiltoniano de coste_) cuyo estado base codifica la solución de nuestro problema.

Partimos de un Hamiltoniano $H_0$ cuyo estado base es facil de preparar y queremos llegar, en un tiempo $T$, al estado base de $H_c$. Hacemos evolucionar el sistema desde $t=0$ hasta $t=T$ como sigue:

$$H(t) = A(t)H_0 + B(t)H_c$$

donde las funciones $A$ y $B$ verifican: $A(0) = B(T) = 1$ y $A(T) = B(0) = 0$.

Un ejemplo de estas funciones son

$$
A(t) =  \left(1-\frac{t}{T}\right)\\
B(t) = \frac{t}{T}
$$


Si $t$ cambia muy lentamente, el sistema pasa de $H(0) = H_0$ a $H(T) = H_c$ manteniéndose en el estado base.

Midiendo el estado del sistema en $t=T$ obtenemos el estado $|\psi\rangle$ que minimiza $\langle\psi|H_c|\psi\rangle$ y que es la solución del problema.

En [Aharonov et al. 2007](https://doi.org/10.1137/080734479) se demostró que la computación cuántica adiabática es equivalente a la computación cuántica basada en puertas.

#### Tiempo de evolución

El teorema adiabático establece que para que $H(t)$ se mantenga en el estado base, $T$ tiene que ser inversamente proporcional al cuadrado del *spectral gap* (minima diferencia de energía entre el estado base y el primer estado excitado de $H(t)$ con $t\in[0,T]$).

Encontrar el valor del *spectral gap* es muy complejo. Además, $T$ podría llegar a ser muy grande. El *Quantum annealing* es una implementación práctica de la computación cuántica adiabática.




<a name="annealing"></a>

## **Quantum annealing**

Es un modelo similar a la computación cuántica adiabática, con ciertas limitaciones:

- El Hamiltoniano final tiene que ser un Hamiltoniano Ising:  

$$
H_c = -\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - \sum_{i=0}^{n-1} h_{i}Z_i
$$

- No se mantiene la evolución adiabática $\implies$ el estado final podría no ser el estado base de $H_c$

**Estado inicial $H_0$**

Se suele usar como estado inicial el siguiente:

$$
H_0 = -\sum_{i=0}^{n-1} X_i
$$

Este estado se denomina _mixing Hamiltonian_ y su estado base es $|\psi_0\rangle = |+\rangle^{\otimes n}$

**Evolución del Hamiltoniano**

La evolución viene dada por:

$$
H(t) = A(t)H_0 + B(t)H_c = -A(t)\sum_{i=0}^{n-1} X_i - B(t)\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - B(t)\sum_{i=0}^{n-1} h_{i}Z_i
$$


### Quantum annealers
Computadores cuánticos que resuelven problemas de optimización mediante Quantum annealing.

- [D-Wave Systems](https://www.dwavesys.com/): Primera empresa en vender quantum annealers
  - El último modelo dispone de más de 7000 cúbits
  - Suite [Ocean](https://docs.ocean.dwavesys.com/en/latest/)
  - Tipo de problemas: optimización de portfolios, optimización de rutas y gestión de flotas, optimización de líneas de producción, búsqueda de similaridad entre moléculas, etc.




In [ ]:
!pip install dwave-neal dimod

In [ ]:
# dimod: biblioteca base de D-Wave Ocean para representar
# y resolver problemas combinatorios binarios cuadráticos
from dimod import BinaryQuadraticModel, SPIN
J = {(0,1):1, (0,2):1}
h = {}

# Expresamos el grafo simple como un modelo binario cuadrático
# SPIN: las variables toman valores {-1,1}
problem = BinaryQuadraticModel(h, J, 0.0, SPIN)
print("The problem we are going to solve is:")
print(problem)

In [ ]:
# neal: simulador de annealing de DWave
from neal import SimulatedAnnealingSampler

# Usamos un simulador clásico en vez del D-WaveSampler
sampler = SimulatedAnnealingSampler()

# Ejecutamos la muestra con 10 repeticiones
result = sampler.sample(problem, num_reads=10)

print("\nThe solutions that we have obtained are:")
print(result)


In [ ]:
# Uso en un sistemna Dwave
# 1. Instalar el SDK de Ocean con pip install dwave-ocean-sdk
# 2. Registrarse en Leap (https://cloud.dwavesys.com/leap/) y obtener una clave API
# 3. Usar "dwave config create" para añadir la clave a nuestra configuración
#
# Código adaptado
#from dwave.system import DWaveSampler
#from dwave.system import EmbeddingComposite
#sampler = EmbeddingComposite(DWaveSampler())
#result = sampler.sample(problem, num_reads=10)
#print("The solutions that we have obtained are")
#print(result)



---



---



---



<a name="vqe"></a>
# **Variational Quantum Eigensolvers (VQE)**

VQE es una generalización de QAOA para aproximar el estado base de un hamiltoniano H genérico. Se basa en el principio variacional, que establece que el mínimo valor esperado de un observable se alcanza siempre en un autovector del mismo. Es decir, que para cualquier estado arbitrario, el valor esperado de un observable es siempre mayor o igual al mínimo.


<details>
  <summary>Pulsa aquí para una explicación del principio variacional</summary>

El principio variacional permite estimar un límite superior para la energía del estado base (_ground state_) de un sistema cuántico.
    
Supongamos un estado cuántico, caracterizado por un hamiltoniano $H$, que representa la energía del sistema. Los posibles valores de energía son los autovalores $\lambda_i$ de la matriz $H$, y la energía del sistema en el estado $\vert u_i\rangle$ viene dada por el valor esperado de $H$ en ese estado:

$$
E(\vert u_i \rangle)\equiv \lambda_i = \langle H\rangle_{|u_i\rangle} = \langle u_i \vert H \vert u_i \rangle
$$


El estado base del sistema es el correspondiente al mínimo de energía. Supongamos que ese mínimo corresponde al autoestado $|u_{min}\rangle$ con autovalor $\lambda_{min}$. Para cualquier autovalor $\lambda_i$ se tiene que:

\begin{align*}
    E_{min} \equiv \lambda_{min} \le \lambda_i \equiv \langle H \rangle_{|u_i\rangle} = \langle u_i |H|u_i \rangle
\end{align*}  

donde $|u_i\rangle$ es el autoestado asociado a $\lambda_i$.

Sea un estado arbitrario $|\psi\rangle$. Escribiendo $H$ en la base de sus autoestados $H = \sum_{i=1}^N \lambda_i |u_i\rangle\langle u_i|$ tenemos que el valor esperado de la energía en el estado $|\psi\rangle$:

\begin{align}
E(|\psi\rangle)=\langle H\rangle_{|\psi\rangle} = \langle \psi |H|\psi \rangle & = \
 \langle \psi | \left( \sum_{i=1}^N \lambda_i |u_i\rangle\langle u_i| \right) |\psi \rangle\\
 &= \sum_{i=1}^N \lambda_i \langle \psi |u_i\rangle\langle u_i|\psi \rangle\\
 &= \sum_{i=1}^N \lambda_i |\langle u_i|\psi \rangle|^2
\end{align}

Es decir, el valor esperado de $H$ en un estado cualquiera es una combinación lineal en la que los autovalores actúan como pesos. De esta expresión es facil ver que:

\begin{align}
    \lambda_{min} \le \langle H \rangle_{|\psi\rangle} = \langle \psi | H | \psi \rangle = \sum_{i = 1}^{N} \lambda_i | \langle u_i | \psi\rangle |^2
\end{align}

Esa expresión se denomina _principio variacional_ y simplemente indica que el valor mínimo de energía es menor o igual que el valor esperado de $H$ en un estado arbitrario.

</details>

VQE se suele usar en problemas de química computacional, por ejemplo, para obtener el mínimo estado de energía de una determinada molécula. También se puede usar en problemas de optimización combinatoria.

VQE aproxima el estado base de un hamiltoniano H. Para ello se crea un circuito cuántico variacional (denominado _forma variacional_ o _ansatz_) en un estado inicial $|\psi\rangle$. Ejecutando el circuito, se obtiene el valor esperado $\langle H \rangle_{|\psi\rangle}$. Se usa un  optimizador clásico para ajustar los parámetros del circuito con con vistas a encontrar los parámetros que minimicen $\langle H \rangle_{|\psi\rangle}$.

<center><img src="https://drive.google.com/uc?export=view&id=1O9RXUnTfu09PLD7O4XT1FrC2GcSE008t" alt="Circuito variacional" width="900"  /></center>


La idea de VQE es, partiendo del estado inicial, ir recorriendo el espacio de estados (o lo que es lo mismo, la esfera de Bloch) y calcular el valor esperado en $H$ en cada estado.

Por ejemplo, se podría elegir como valor inicial el estado $|0\rangle$ e ir aplicando rotaciones $R_Y$ o $R_x$:

<center><img src="https://drive.google.com/uc?export=view&id=1y94qu2xOloSXoTe-8t9nwgehEvgfZHG1" alt="VQE" width="200"  /></center>
(Fuente: <a href=https://www.mustythoughts.com/variational-quantum-eigensolver-explained>https://www.mustythoughts.com/variational-quantum-eigensolver-explained</a>)

En general, se usan ansätze $V(\theta)$ más complejos que permitan recorrer la esfera de Bloch.

#### Optimización de los parámetros

El algoritmo VQE parte de un Hamiltoniano $H$, cuyo estado base resuelve el problema, y procede como sigue:

1. Se elige un estado inicial (normalmente $|\psi(\theta)\rangle = |0\rangle$) y un _ansatz_ $V(\theta)$
2. Se le dan valores iniciales a los parámetros $\theta$
2. En la QPU se ejecuta el ansatz con esos parámetros para obtener un estado $|\psi(\theta)\rangle = V(\theta)|0\rangle$
3. Se mide el valor esperado del Hamiltoniano en ese estado $\langle\psi(\theta)|H|\psi(\theta)\rangle$
4. En la CPU se usa un algoritmo de optimización para modificar los parámetros del ansatz
    - Objetivo: reducir el valor esperado del Hamiltoniano
5. Se vuelve al paso 3 hasta que se alcanza un mínimo

Igual que en QAOA, el proceso de optimización de los parámetros puede ser un simple gradiente descendente, en el que cada parámetro se actualiza en la dirección que conduzca al mayor descenso de la energía, u otros método de optimización más sofisticados.

#### Obtención del valor esperado

A diferencia del QAOA, en el que el Hamiltoniano es una matriz diagonal, en VQE es más general $\implies$ no es trivial obtener el valor esperado.

Solución: Expresar el Hamiltoniano como suma de productos tensor de matrices de Pauli y obtener el valor esperado tomando medidas en diferentes bases.

#### Ejemplos de ansätze

La elección de la forma variacional o ansatz para VQE depende del problema a tratar. Un ejemplo es _RealAmplitudes_ que vimos antes. Otra forma es la _EfficientSU2_.

In [ ]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement='linear', insert_barriers=True)

ansatz.decompose().draw('mpl')

### Obtención de estados excitados

El algoritmo VQD (_Variational Quantum Deflaction_) es una extensión del VQE que permite obtener _estados excitados_, es decir, autoestados con mayor energía.

Para ello, si $|\psi_0\rangle$ es el estado base, es posible demostrar que el estado base del siguiente Hamiltoniano:

$$
H' = H + C|\psi_0\rangle\langle\psi_0|
$$
es el primer estado excitado de H

#### Referencias:

  - Peruzzo, et al. (2014). A variational eigenvalue solver on a photonic quantum processor. Nature communications, 5(1), 1-7. [arXiv:1304.3061](https://arxiv.org/abs/1304.3061)
  - Higgott, O., Wang, D., & Brierley, S. (2019). Variational quantum computation of excited states. Quantum, 3, 156. [arXiv:1805.08138](https://arxiv.org/abs/1805.08138)
  - Combarro, E.F. & Gonzákez-Castillo, S. (2023). A Practical Guide  to Quantum Machine Learning and Quantum Optimization, capítulo  7. Packt.
  - Qiskit tutorial: Variational quantum eigensolver https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver




---



---



---

